In [1]:
import pandas as pd

/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_16441/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### SQL

In [313]:
"""select u.username, u.name, ua.*, uar.reason from kjtcallctr.useraction ua 
left join kjt.useractionreason uar on uar.id= ua.reasonId
left join kjt.user u on u.uid = ua.uid 
where ua.createTime > '2024-03-10'
and u.name like '%Irish%'
order by createTime;"""

"select u.username, u.name, ua.*, uar.reason from kjtcallctr.useraction ua \nleft join kjt.useractionreason uar on uar.id= ua.reasonId\nleft join kjt.user u on u.uid = ua.uid \nwhere ua.createTime > '2024-03-10'\nand u.name like '%Irish%'\norder by createTime;"

# Script

In [948]:
df = pd.read_csv('pos_data_snowflake.csv',parse_dates=['CREATETIME','ENDTIME'],infer_datetime_format=True)
df.columns = [x.lower() for x in df.columns]

df.createtime = df.createtime.dt.tz_localize(None)
df.endtime = df.endtime.dt.tz_localize(None)

/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_16441/2774530968.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('pos_data_snowflake.csv',parse_dates=['CREATETIME','ENDTIME'],infer_datetime_format=True)


In [949]:
df.rename(columns={'endtime': 'end_interval_log'}, inplace=True)
df.rename(columns={'createtime': 'start_interval_log'}, inplace=True)

In [950]:
df['reason'] = df['reason'].fillna("Phone Time")

In [951]:
df.rename(columns={'reason': 'pos_status'}, inplace=True)

In [958]:
df = df.loc[df.start_interval_log <  df.end_interval_log]

In [1392]:
dx = pd.read_excel('report (23).xlsx')

/Users/wonders/Documents/python/paydispute/venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [1393]:
dx.columns = range(len(list(dx)))

#### match names

In [1394]:
from datetime import datetime 

In [1401]:
dx.iloc[126327:].iloc[:1000].to_clipboard()

In [1402]:
agent_name = None
current_date = None

agent_names = []
iex_numbers = []
current_dates = []
pos_statuses = []
start_times = []
end_times = []


for i,row in dx.iloc[1:].iterrows():

    #check if new agent
    if pd.notna(row[1]):
        if row[1][:5] == 'Agent':

            agent_name = " ".join(row[1].split(':')[1].split()[1:])
            iex_number = row[1].split(':')[1].split()[0]
        else:
            pass

    #check if new date
    if pd.notna(row[2]):
        try:
            current_date = pd.to_datetime(row[2])
        except:
            pass

    #add entry
    if pd.notna(row[6]):
        agent_names.append(agent_name)
        iex_numbers.append(iex_number)
        current_dates.append(current_date)
        pos_statuses.append(row[6])
        start_times.append(row[7])
        end_times.append(row[10])


In [1403]:
dx = pd.DataFrame({'agent_names':agent_names,'iex_no':iex_numbers,'date':current_dates,'schedule':pos_statuses,'start_time':start_times,'end_time':end_times})

In [1406]:
dx.loc[dx.agent_names.str.contains('Puntero')].to_clipboard()

#### match_agent

In [47]:
name_key = {}

In [48]:
for agent in dx.agent_names.unique().tolist():
    matches = process.extractOne(agent, names_list)
    name_key[agent] = matches

In [52]:
name_df = pd.DataFrame(name_key).T
name_df.columns = ['pos_name','match_score']

In [58]:
name_df['positive_match'] = name_df.match_score.apply(lambda x: x > 86)

In [332]:
def start_interval(date,start_time):
    date_obj = datetime(date.year, date.month, date.day, start_time.hour, start_time.minute,start_time.second)
    timestamp = pd.to_datetime(date_obj)
    return timestamp




In [645]:
df1 = dx.copy()

In [646]:
df1 = pd.merge(df1,name_df.loc[name_df.positive_match,['pos_name']],left_on= 'agent_names', right_index=True,how='left')

In [647]:
df1.start_time = pd.to_datetime(df1.start_time)
df1.end_time = pd.to_datetime(df1.end_time)

/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_16441/341441131.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1.start_time = pd.to_datetime(df1.start_time)
/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_16441/341441131.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df1.end_time = pd.to_datetime(df1.end_time)


In [648]:
df1['start_interval_sched'] = df1.apply(lambda row: start_interval(row.date,row.start_time),axis=1)
df1['end_interval_sched'] = df1.apply(lambda row: start_interval(row.date,row.end_time),axis=1)

##### KEY

1.  Happy
2.  Benefit of the doubt
3. Working When you shouldn't be 
4.  Not Working when you should be 
5. working when you REALLY shouldn't be 
6. Overtime on break
7. Unverified Overtime
8. Happy not working
99. Phone Time == Phone Time

In [970]:
activity_matrix = {
        "Phone Time": {"Phone Time": 99, "Pullout": 2, "Other": 2, "Break": 4, "Restroom": 4, "Standby": 4, "Forced": 4, "Logout": 4,"Meeting":2},
        "Pullout": {"Phone Time": 98, "Pullout": 1, "Other": 1, "Break": 4, "Restroom": 4, "Standby": 2, "Forced": 4, "Logout": 4,"Meeting":2},
        "Overtime": {"Phone Time": 97, "Pullout": 2, "Other": 2, "Break": 6, "Restroom": 6, "Standby": 2, "Forced": 6, "Logout": 6,"Meeting":2},
        "Rest Day Overtime": {"Phone Time": 97, "Pullout": 2, "Other": 2, "Break": 6, "Restroom": 6, "Standby": 2, "Forced": 6, "Logout": 6,"Meeting":2},
        "Break": {"Phone Time": 98, "Pullout": 3, "Other": 3, "Break": 8, "Restroom": 8, "Standby": 8, "Forced": 8, "Logout": 8, "Meeting":3},
        "Scheduled Off": {"Phone Time": 7, "Pullout": 7, "Other": 7, "Break": 8, "Restroom": 8, "Standby": 8, "Forced": 8, "Logout": 8,"Meeting":5},
        "Paid Vacation Leave": {"Phone Time": 5, "Pullout": 5, "Other": 5, "Break": 8, "Restroom": 8, "Standby": 8, "Forced":8, "Logout": 8,'Meeting':5},
        "Absent-Unpaid": {"Phone Time": 5, "Pullout": 5, "Other": 5, "Break": 8, "Restroom": 8, "Standby": 8, "Forced": 8, "Logout": 8,"Meeting":5},
    }

In [1410]:
pd.DataFrame(activity_matrix).to_clipboard()

#### from here, analyze per agent

In [916]:
def create_scheduled_off(tdx):
    scheduled_offs = []
    for i,row in tdx.loc[tdx.time_between_sched > 0].iterrows():
        row.end_interval_sched = row.start_interval_sched
        row.start_interval_sched = row.last_end_interval_sched
        row.pos_status = 'Scheduled Off'
        scheduled_offs.append(row)
    
    return pd.DataFrame(scheduled_offs)

In [1011]:
def log_exceeds_schedule(dfdirty):
    added_rows = []
    for i,row in dfdirty.iloc[:].iterrows():
        entry1 = row.copy()
        entry1.end_interval_log = entry1.end_interval_sched
        added_rows.append(entry1)
        entry2 = row.copy()
        entry2.start_interval_log = entry2.end_interval_sched
        added_rows.append(entry2)

    df_new = pd.DataFrame(added_rows)
    return df_new


In [1028]:
def fix_split_interval(tdf,tdx):
    tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', right_on='start_interval_sched',suffixes=('_log', '_sched'), direction='backward')
    tdf1 = tdf1[['name','team_name','type','schedule','pos_status','start_interval_log','end_interval_log','start_interval_sched','end_interval_sched']]
    tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log
    dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.schedule))]
    dfdirty = tdf1.loc[tdf1.end_surpass]
    
    for i in [1,2,3]:
        df_new = log_exceeds_schedule(dfdirty)
        df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log
        df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log
        dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])
        dfdirty = df_new.loc[df_new.end_surpass]

    dfclean['activity_type'] = dfclean.apply(lambda row: activity_matrix.get(row.schedule).get(row['pos_status']),axis=1)
    dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]
    return dfclean

In [1110]:
def create_agent_timesheet(agent_name,df=df):
    tdx = df1.loc[df1.agent_names == agent]
    tdx = tdx.sort_values(by='start_interval_sched').reset_index(drop=True)
    tdx['last_end_interval_sched']  = tdx['end_interval_sched'].shift(1)
    tdx['time_between_sched'] = [x.seconds / 60 / 60 for x in tdx.start_interval_sched - tdx['end_interval_sched'].shift(1) ]
    offs = create_scheduled_off(tdx)
    tdx = pd.concat([tdx,offs])
    tdx = tdx.sort_values(by='start_interval_sched',ascending=True)

    
    tdf = df.loc[df.name == agent_name]
    dfclean  =  fix_split_interval(tdf,tdx)
    return dfclean

In [1080]:
create_agent_timesheet("Ron Loyola",df).to_clipboard()

In [1085]:
dx.loc[dx.agent_names == "Loyola, Ron"].to_clipboard()

In [1000]:
df.loc[df.name == "Ron Loyola"]

,username,team_name,name,type,start_interval_log,end_interval_log,pos_status
12044,ronloyola,Murfreesboro,Ron Loyola,off,2024-03-01 14:30:20,2024-03-01 14:30:41,Other
12065,ronloyola,Murfreesboro,Ron Loyola,on,2024-03-01 14:30:41,2024-03-01 15:15:01,Phone Time
14249,ronloyola,Murfreesboro,Ron Loyola,off,2024-03-01 15:15:01,2024-03-01 16:00:20,Break
16913,ronloyola,Murfreesboro,Ron Loyola,on,2024-03-01 16:00:20,2024-03-01 16:07:40,Phone Time
17380,ronloyola,Murfreesboro,Ron Loyola,off,2024-03-01 16:07:40,2024-03-01 16:09:12,Restroom
...,...,...,...,...,...,...,...
859890,ronloyola,Murfreesboro,Ron Loyola,off,2024-03-18 17:48:11,2024-03-18 18:03:58,Break
860595,ronloyola,Murfreesboro,Ron Loyola,on,2024-03-18 18:03:58,2024-03-18 20:46:05,Phone Time
867668,ronloyola,Murfreesboro,Ron Loyola,off,2024-03-18 20:46:05,2024-03-18 21:01:33,Break
868335,ronloyola,Murfreesboro,Ron Loyola,on,2024-03-18 21:01:33,2024-03-18 22:00:02,Phone Time


In [1094]:
df = df.loc[df.start_interval_log < '2024-03-17']

In [1166]:
agent_list = df1.groupby(['pos_name','agent_names']).first().index.tolist()

In [1179]:
total_df = pd.DataFrame()

In [1180]:
for a in agent_list[:]:
    if len(df.loc[df.name == a[0]]) < 1:
        not_working.append(a[0])
        print('woops {}'.format(a[0]))
        continue
    try:
        total_df = pd.concat([total_df,create_agent_timesheet(a[0])])
    except:
        not_working.append(a)

woops AR Abdulazeez
woops Ailin Li
woops Alexis Jara
woops Amado Padron
woops Andrea Luna
woops Angela Loyola Online
woops Arnel Terio
woops Bev Lozanes
woops Bobby Thai
woops Carl Perez
woops Charm Cabrillos
woops Charmaine Saguran
woops Christian Obando
woops Cindy Singson
woops Claire Refe
woops Coleen Yucor
woops Crystel Layoso
woops Dan Umandal
woops David  Dreyfus
woops Dragones Bungcasan
woops Ethan Cai
woops Fil Bangsal
woops Hernan Valdez
woops Irene Alo
woops Isaac Perez
woops Ivy Chiong
woops Jane Kilapkilap
woops Jay Penaflor
woops Jayson Zanarias
woops Jc Perez
woops Jennifer Aguila
woops Jeram Sebuco
woops Jessica Aye
woops Jiaxin Chen
woops Jim Santiano
woops Joe Moriarty
woops Joey Xiao
woops John Mirasol
woops John Perez
woops John Villaruel
woops Jonna Ocampo Online
woops Joy Lin
woops June Sisican
woops Juver Voluntad
woops Karol Gomez
woops Kevin Diaz
woops Krisha Torres
woops Liz Delfin
woops Lu Li
woops Ly Tibunsay
woops MJ Camasis
woops Manuel Jimenez
woops Marc 

In [1183]:
total_df.groupby(['activity_type'])['mins'].sum() /total_df.groupby(['activity_type'])['mins'].sum().sum()

activity_type
1.0     0.000494
2.0     0.106349
3.0     0.009035
4.0     0.198173
5.0     0.139305
8.0     0.069860
98.0    0.030675
99.0    0.446109
Name: mins, dtype: float64

In [ ]:
total_df = total_df.loc[~total_df.team_name.isin(remove_team)]

In [1186]:
total_df.groupby(['name','team_name','activity_type'])[['mins']].sum().to_clipboard()

In [1189]:
total_df.loc[total_df.name == 'Cath Acebuche'].to_clipboard()

In [1205]:
len(total_df.name.unique())

1120

In [1391]:
total_df.loc[total_df.name.str.contains("Puntero")].to_clipboard()

In [ ]:
# Agents Scheduled for Phone Time
# Agents On Phone Time 


In [1356]:
check_time = '2024-03-08 16:45'

In [1360]:
    cri_a = total_df.start_interval_sched < check_time 
    cri_b = total_df.end_interval_sched > check_time 
    cri_status = total_df.schedule == 'Phone Time'

In [1367]:
def check_phone_time(check_time):
    cri_a = total_df.start_interval_log < check_time 
    cri_b = total_df.end_interval_log > check_time 
    cri_status = total_df.pos_status == 'Phone Time'
    phone_time = len(total_df.loc[cri_a & cri_b & cri_status])
    
    cri_a = total_df.start_interval_sched < check_time 
    cri_b = total_df.end_interval_sched > check_time 
    cri_status = total_df.schedule == 'Phone Time'
    sched_time = len(total_df.loc[cri_a & cri_b & cri_status].name.unique())
    return (phone_time, sched_time)




def check_phone_time_roster(check_time):
    cri_a = total_df.start_interval_log < check_time 
    cri_b = total_df.end_interval_log > check_time 
    
    cri_scheduled = total_df.schedule == 'Phone Time'
    cri_status = total_df.pos_status == 'Phone Time'

    return (total_df.loc[cri_a & cri_b & cri_status],total_df.loc[cri_a & cri_b & cri_scheduled])

In [1377]:
check_phone_time('2024-03-01 10:30:00')

(0, 0)

In [1378]:
import pandas as pd

# Define start date and time
start_datetime = '2024-03-08 10:30'

# Create a DatetimeIndex starting from start_datetime with a frequency of every 5 minutes
date_series = pd.date_range(start=start_datetime, periods=5000, freq='5T')

print(date_series)


DatetimeIndex(['2024-03-08 10:30:00', '2024-03-08 10:35:00',
               '2024-03-08 10:40:00', '2024-03-08 10:45:00',
               '2024-03-08 10:50:00', '2024-03-08 10:55:00',
               '2024-03-08 11:00:00', '2024-03-08 11:05:00',
               '2024-03-08 11:10:00', '2024-03-08 11:15:00',
               ...
               '2024-03-25 18:20:00', '2024-03-25 18:25:00',
               '2024-03-25 18:30:00', '2024-03-25 18:35:00',
               '2024-03-25 18:40:00', '2024-03-25 18:45:00',
               '2024-03-25 18:50:00', '2024-03-25 18:55:00',
               '2024-03-25 19:00:00', '2024-03-25 19:05:00'],
              dtype='datetime64[ns]', length=5000, freq='5min')


/var/folders/q2/1vhx3pcx0h37hzlw8pvc0s380000gn/T/ipykernel_16441/1183041946.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_series = pd.date_range(start=start_datetime, periods=5000, freq='5T')


In [1379]:
dfc = pd.DataFrame(date_series)

In [1380]:
res = []

In [1381]:
for i in dfc[0].tolist():
    res.append(check_phone_time(i))

In [1382]:
dfc['on_phone'] = [x[0] for x in res]
dfc['scheduled'] = [x[1] for x in res]

In [1385]:
dfc

,0,on_phone,scheduled
0,2024-03-08 10:30:00,0,0
1,2024-03-08 10:35:00,62,438
2,2024-03-08 10:40:00,62,438
3,2024-03-08 10:45:00,66,438
4,2024-03-08 10:50:00,123,438
...,...,...,...
4995,2024-03-25 18:45:00,0,0
4996,2024-03-25 18:50:00,0,0
4997,2024-03-25 18:55:00,0,0
4998,2024-03-25 19:00:00,0,0


In [1386]:
dfc.to_clipboard()

In [1291]:
total_df.loc[total_df.name == 'Jun Reyes'].to_clipboard()

In [126]:
tdx = df1.loc[df1.agent_names == agent]
tdx = tdx.sort_values(by='start_interval_sched').reset_index(drop=True)
tdx['last_end_interval_sched']  = tdx['end_interval_sched'].shift(1)
tdx['time_between_sched'] = [x.seconds / 60 / 60 for x in tdx.start_interval_sched - tdx['end_interval_sched'].shift(1) ]


In [205]:
tdf = df.loc[df.name == agent_pos_key]

#### create SCHEDULED_OFF 

In [206]:
tdx.pos_status.unique().tolist()

['Phone Time', 'Break', 'Scheduled Off', 'Absent-Unpaid', 'Pullout']

In [275]:
def log_exceeds_schedule(dfdirty):
    added_rows = []
    for i,row in dfdirty.iloc[:].iterrows():
        entry1 = row.copy()
        entry1.end_interval_log = entry1.end_interval_sched
        added_rows.append(entry1)
        entry2 = row.copy()
        entry2.start_interval_log = entry2.end_interval_sched
        added_rows.append(entry2)

    df_new = pd.DataFrame(added_rows)[['name','uid','type','reason.1','start_interval_log','end_interval_log']]
    return df_new


In [276]:
def fix_split_interval():
    tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', right_on='start_interval_sched',suffixes=('_log', '_sched'), direction='backward')
    tdf1 = tdf1[['name','uid','type','reason.1','pos_status','start_interval_log','end_interval_log','start_interval_sched','end_interval_sched']]
    tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log
    dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.pos_status))]

    dfdirty = tdf1.loc[tdf1.end_surpass]
    for i in [1,2,3]:
        df_new = log_exceeds_schedule(dfdirty)
        dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])
        dfdirty = df_new.loc[df_new.end_surpass]

    dfclean['activity_type'] = dfclean.apply(lambda row: activity_matrix.get(row.pos_status).get(row['reason.1']),axis=1)
    dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]
    dfclean[['name','reason.1','start_interval_log','end_interval_log','pos_status','start_interval_sched','end_interval_sched','mins','activity_type']].to_clipboard(index=False)

    return dfclean

#### find split Interval

In [214]:
tdf1 = pd.merge_asof(tdf.sort_values(by='start_interval_log',ascending=True),tdx.sort_values(by='start_interval_sched',ascending=True),
    left_on='start_interval_log', 
    right_on='start_interval_sched',
    suffixes=('_log', '_sched'), 
    direction='backward'
)

In [215]:
tdf1 = tdf1[['name','uid','type','reason.1','pos_status','start_interval_log','end_interval_log','start_interval_sched','end_interval_sched']]

In [217]:
tdf1['end_surpass'] = tdf1.end_interval_sched < tdf1.end_interval_log

In [218]:
dfclean = tdf1.loc[(tdf1.end_surpass == False) & (~pd.isna(tdf1.pos_status))]
len(dfclean)

334

In [220]:
dfdirty = tdf1.loc[tdf1.end_surpass]
len(dfdirty)

153

In [250]:
dfclean['activity_type'] = dfclean.apply(lambda row: activity_matrix.get(row.pos_status).get(row['reason.1']),axis=1)

In [251]:
dfclean['mins'] = [round(x.seconds / 60,2)  for x in dfclean.end_interval_log - dfclean.start_interval_log]

In [253]:
dfclean.to_clipboard()

In [252]:
dfclean.groupby('activity_type')['mins'].sum()

activity_type
1    12722.96
2     6507.86
3        8.47
4    14405.37
5        0.05
7        4.21
8    29395.45
Name: mins, dtype: float64

In [227]:
dfclean[['name','reason.1','start_interval_log','end_interval_log','pos_status','start_interval_sched','end_interval_sched','mins','activity_type']].to_clipboard(index=False)

#### the problem is the end
1. make a new entry
2. clean up entry 

In [238]:
added_rows = []
for i,row in dfdirty.iloc[:].iterrows():
    entry1 = row.copy()
    entry1.end_interval_log = entry1.end_interval_sched
    added_rows.append(entry1)
    entry2 = row.copy()
    entry2.start_interval_log = entry2.end_interval_sched
    added_rows.append(entry2)


In [239]:
df_new = pd.DataFrame(added_rows)

In [240]:
df_new = df_new[['name','uid','type','reason.1','start_interval_log','end_interval_log']]

In [241]:
df_new = pd.merge_asof(df_new.sort_values(by='start_interval_log'),tdx[['pos_status','start_interval_sched','end_interval_sched']],
    left_on='start_interval_log', 
    right_on='start_interval_sched',
    suffixes=('_log', '_sched'), 
    direction='backward'
)

In [242]:
df_new['end_surpass'] = df_new.end_interval_sched < df_new.end_interval_log

In [243]:
df_new.groupby('end_surpass').size()

end_surpass
False    74
True     68
dtype: int64

In [244]:
dfclean = pd.concat([dfclean,df_new.loc[df_new.end_surpass==False]])

In [245]:
dfdirty = df_new.loc[df_new.end_surpass]
len(dfdirty)

68

In [ ]:
def is_overlap(start1, end1, start2, end2):
    return max(start1, start2) < min(end1, end2)

In [ ]:
def get_intersection_and_non_overlap(schedule_start, schedule_end, actual_intervals):
    intersection_intervals = []
    non_overlap_intervals = []
    
    # Initial non-overlapped interval is the whole schedule
    non_overlap_intervals.append((schedule_start, schedule_end))
    
    for actual_start, actual_end in actual_intervals:
        new_non_overlap_intervals = []
        
        for non_overlap_start, non_overlap_end in non_overlap_intervals:
            # Check for overlap
            if actual_end < non_overlap_start or actual_start > non_overlap_end:
                # No overlap, keep the interval as is
                new_non_overlap_intervals.append((non_overlap_start, non_overlap_end))
            else:
                # If there's overlap, adjust non-overlapped intervals and record intersection
                if actual_start > non_overlap_start:
                    new_non_overlap_intervals.append((non_overlap_start, actual_start))
                if actual_end < non_overlap_end:
                    new_non_overlap_intervals.append((actual_end, non_overlap_end))
                
                # Add to intersection intervals
                intersection_start = max(non_overlap_start, actual_start)
                intersection_end = min(non_overlap_end, actual_end)
                intersection_intervals.append((intersection_start, intersection_end))
        
        # Update non-overlap intervals after processing this actual interval
        non_overlap_intervals = new_non_overlap_intervals

    non_compliance_seconds = 0

    for interval in non_overlap_intervals:
        non_compliance_seconds += (interval[1] - interval[0]).seconds 
    
    return intersection_intervals, non_overlap_intervals,non_compliance_seconds





In [ ]:
tdx.start_time = pd.to_datetime(tdx.start_time)
tdx.end_time = pd.to_datetime(tdx.end_time)

In [ ]:
tdx['start_interval'] = tdx.apply(startInterval,axis=1)
tdx['start_interval'] = [x.tz_localize('UTC') for x in tdx.start_interval]
tdx['end_interval'] = tdx.apply(endInterval,axis=1)
tdx['end_interval'] = [x.tz_localize('UTC') for x in tdx.end_interval]


In [ ]:
def get_non_compliance_seconds(row):
    if row.pos_status != 'Phone Time':
        return 0
    else:
        return get_intersection_and_non_overlap(row.start_interval, row.end_interval, [(row.createtime,row.endtime) for i,row in tdf.iterrows()])[2]


def get_non_compliance_seconds_stretch(row):
    if row.pos_status != 'Phone Time':
        return 0
    else:
        return get_intersection_and_non_overlap(row.start_interval, row.end_interval, [(row.createtime,row.endtime) for i,row in temp_df2.iterrows()])[2]
        

In [ ]:
tdf = tdf.loc[tdf.type == 'on']

In [ ]:
tdx['non_adherance'] = tdx.apply(lambda row: get_non_compliance_seconds(row) if row.pos_status == 'Phone Time' else 0,axis=1)

In [ ]:
tdx['non_adherance_inc'] = tdx.apply(lambda row: get_non_compliance_seconds_stretch(row) if row.pos_status == 'Phone Time' else 0,axis=1)

In [ ]:
tdx['delta'] = tdx.non_adherance - tdx.non_adherance_inc

In [ ]:
tdx.loc[tdx.pos_status == 'Phone Time'].sort_values(by='delta',ascending=False)['non_adherance'].sum() / 60 / 60

In [ ]:
tdx.loc[tdx.pos_status == 'Phone Time'].sort_values(by='delta',ascending=False)

In [ ]:
tdf